In [ ]:
# Pytorch version Compatible with mmcv prebuilt wheels
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# Transformers version to match

!pip install -q transformers==4.30.2

# Fast MMPose Setup for Colab Pro (no manual wheel building)
!pip install -q -U openmim
!mim install mmengine
!mim install "mmdet==3.0.0"
!mim install "mmcv==2.0.0"  # No wheel build — fast precompiled binary

# Ultralytics install
!pip install ultralytics

# Downgrade to stable NumPy

!pip install numpy==1.26.4

# Gradio install for gui through Colab
!pip install gradio

#install dynamic time warping function for analysis phase
!pip install dtw

# stable matplotlib version that works with mmpose functions

!pip install --upgrade matplotlib==3.5.2



In [ ]:
# Must restart colab session for package versions to update
import os
os.kill(os.getpid(), 9)

In [ ]:
# MMPose install
%cd  /content/ComputingProject/mmpose
!pip install -e .

In [ ]:
import pickle
import numpy as np
import cv2
from ultralytics import YOLO
from mmpose.apis import visualize, MMPoseInferencer
import matplotlib.pyplot as plt
import torch
import transformers
import mmcv
import mmdet
import mmengine
import mmpose
from tqdm import tqdm
import math
from dtw import dtw
from PIL import Image
import tempfile
import shutil
import gradio as gr


In [ ]:
def lock_shooter(video_path, yolo_model, mmpose_inferencer, ball_class_id=0, conf_thresh=0.6, scan_frames=100):
    """
    Lock onto the shooter and return only the visualisation (without side-by-side original image).
    Falls back to using the only detected person if no ball is found.
    """
    cap = cv2.VideoCapture(video_path)
    frame_id = 0

    while frame_id < scan_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        print(f"🔍 Scanning frame {frame_id}")

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_copy = frame_rgb.copy()

        # YOLOv11 ball detection
        results = yolo_model(frame)[0]
        boxes = results.boxes

        ball_boxes = [
            boxes[i] for i in range(len(boxes))
            if boxes.conf[i] > conf_thresh and int(boxes.cls[i]) == ball_class_id
        ]

        # MMPose detection
        pose_result = next(mmpose_inferencer(frame_rgb, return_vis=False))
        pred_instances = pose_result['predictions'][0]
        if not pred_instances:
            continue  # no people found

        # PRIMARY: Use ball if found
        if ball_boxes:
            ball_box = ball_boxes[0]
            x1, y1, x2, y2 = ball_box.xyxy[0].tolist()
            ball_center = np.array([(x1 + x2) / 2, (y1 + y2) / 2])

            min_dist = float('inf')
            shooter_idx = None
            shooter_bbox = None

            for idx, person in enumerate(pred_instances):
                keypoints = person['keypoints']
                left_wrist = keypoints[9][:2]
                right_wrist = keypoints[10][:2]

                dist_left = np.linalg.norm(left_wrist - ball_center)
                dist_right = np.linalg.norm(right_wrist - ball_center)
                min_person_dist = min(dist_left, dist_right)

                if min_person_dist < min_dist:
                    min_dist = min_person_dist
                    shooter_idx = idx
                    shooter_bbox = person['bbox'][0]

            if shooter_bbox:
                print(f"✅ Shooter locked at frame {frame_id} (via ball)")
                cap.release()
                return frame_id, shooter_bbox

        # FALLBACK: Only one person -> assume shooter
        elif len(pred_instances) == 1:
            shooter_data = pred_instances[0]
            shooter_bbox = shooter_data['bbox'][0]
            print(f"✅ Shooter locked at frame {frame_id} (fallback: only one person)")
            cap.release()
            return frame_id, shooter_bbox

    cap.release()
    print("❌ Could not lock shooter in first N frames.")
    return (None, None)

In [ ]:
def calculate_iou(box1, box2):
    # Calculates overlappng bounding box area for tracking
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    width_inter = max(0, x2_inter - x1_inter)
    height_inter = max(0, y2_inter - y1_inter)
    area_inter = width_inter * height_inter

    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    area_union = area_box1 + area_box2 - area_inter

    return area_inter / area_union if area_union > 0 else 0

In [ ]:
def track_shooter_full_video(video_path, yolo_model, mmpose_inferencer, ball_class_id=0, conf_thresh=0.6, scan_frames=100):
    """
    Track the shooter through the entire video and collect keypoint data.

    Args:
        video_path: Path to the input video file
        yolo_model: Loaded YOLO model
        mmpose_inferencer: Loaded MMPose inferencer
        ball_class_id: Class ID for basketball in YOLO
        conf_thresh: Confidence threshold for detections
        scan_frames: Number of frames to scan initially to find shooter

    Returns:
        shooter_keypoints_data: Dictionary with:
        {
            frame_num: {
                'bbox': [x1, y1, x2, y2],
                'keypoints': np.array of shape (N, 3)  # (x, y, score)
                'keypoint_scores': np.array of shape (N,)
                'ball_bbox': [x1, y1, x2, y2] or None
            }
        }
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video {video_path}")

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Find initial shooter
    print("🔍 Finding initial shooter...")
    start_frame, shooter_bbox = lock_shooter(
        video_path, yolo_model, mmpose_inferencer,
        ball_class_id, conf_thresh, scan_frames,
    )

    if start_frame is None:
        cap.release()
        raise RuntimeError("Could not find shooter in initial frames")

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_num = start_frame

    # Data storage
    shooter_keypoints_data = {}
    pbar = tqdm(total=total_frames - start_frame, desc="Tracking shooter")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_data = {
            'bbox': None,
            'keypoints': None,
            'keypoint_scores': None,
            'ball_bbox': None
        }

        # 1. YOLO ball detection
        results = yolo_model(frame)[0]
        boxes = results.boxes
        ball_boxes = [
            boxes[i] for i in range(len(boxes))
            if boxes.conf[i] > conf_thresh and int(boxes.cls[i]) == ball_class_id
        ]

        if ball_boxes:
            ball_box = ball_boxes[0]
            frame_data['ball_bbox'] = ball_box.xyxy[0].tolist()

        # 2. MMPose detection
        pose_result = next(mmpose_inferencer(frame_rgb, return_vis=False))
        pred_instances = pose_result['predictions'][0]

        if pred_instances:
            # Find person with highest IoU with previous shooter bbox
            max_iou = 0
            current_shooter_bbox = None
            shooter_idx = None

            for idx, person in enumerate(pred_instances):
                current_bbox = person['bbox'][0]
                iou = calculate_iou(shooter_bbox, current_bbox)

                if iou > max_iou:
                    max_iou = iou
                    current_shooter_bbox = current_bbox
                    shooter_idx = idx

            # Update tracking if good match found
            if current_shooter_bbox is not None and max_iou >= 0.1:
                shooter_bbox = current_shooter_bbox
                frame_data['bbox'] = shooter_bbox
                frame_data['keypoints'] = np.array(pred_instances[shooter_idx]['keypoints'])
                frame_data['keypoint_scores'] = np.array(pred_instances[shooter_idx].get('keypoint_scores', []))

        shooter_keypoints_data[frame_num] = frame_data
        frame_num += 1
        pbar.update(1)

    pbar.close()
    cap.release()
    print("✅ Tracking complete.")
    return shooter_keypoints_data

In [ ]:
def calculate_knee_angle(keypoints, hip_idx=11, knee_idx=13, ankle_idx=15):
    """Calculates knee angle (hip-knee-ankle) in degrees"""
    hip = keypoints[hip_idx][:2]
    knee = keypoints[knee_idx][:2]
    ankle = keypoints[ankle_idx][:2]

    # Vectors from knee to hip and ankle
    vec_hk = np.array(hip) - np.array(knee)
    vec_ak = np.array(ankle) - np.array(knee)

    # Calculate angle
    cosine = np.dot(vec_hk, vec_ak) / (np.linalg.norm(vec_hk) * np.linalg.norm(vec_ak))
    angle = np.degrees(np.arccos(np.clip(cosine, -1, 1)))
    return angle

In [ ]:
def find_load_frame(frames_dict, release_frame, y_tolerance=5 , lefty=False):
    """
    Finds load frame by working backwards from release until elbow crosses shoulder.

    y_tolerance: Allowed pixels elbow can be below shoulder

    Returns:
        int: Frame number of load position or None
    """
    # Switching joints for left handed shooters
    if lefty:
        shoulder_idx, elbow_idx, wrist_idx = 5, 7, 9
    else:
        shoulder_idx, elbow_idx, wrist_idx = 6, 8, 10

    load_frame = None

    # Work backwards from frame before release
    for frame_num in range(release_frame-1, 0, -1):
        if frame_num not in frames_dict:
            continue

        kps = frames_dict[frame_num]['keypoints']

        # Get required joints (ensure they exist)
        if any(idx >= len(kps) for idx in [shoulder_idx, elbow_idx]):
            continue

        shoulder_y = kps[shoulder_idx][1]
        elbow_y = kps[elbow_idx][1]

        # Check elbow is at/above shoulder (with tolerance)
        if elbow_y <= shoulder_y + y_tolerance:
            load_frame = frame_num
            # Continue searching backwards in case we find earlier load
        else:
            # Stop when we exit the load phase
            if load_frame is not None:
                break
    load_angle = calculate_elbow_angle(kps, shoulder_idx, elbow_idx, wrist_idx)
    knee_angle = calculate_knee_angle(kps)
    return load_frame , load_angle, knee_angle

In [ ]:
def find_preparation_frame(frames_dict, load_frame, still_frame_thresh=3, lefty=False):
    """
    Detects the preparation start frame by tracking upward wrist motion.
    Works backwards from the load frame until wrist hasn't moved upward
    for a number of frames (indicating stillness before shot begins).


    still_frame_thresh: How many frames of no upward wrist movement are required to count as 'stillness'


    Returns:
        int: Frame number where preparation phase begins
    """

    # Use dominant wrist based on handedness (COCO indices)
    wrist_idx = 9 if lefty else 10
    prep_frame = load_frame - 1
    no_upward_motion = 0
    prev_y = None

    for frame_num in range(load_frame - 2, 0, -1):
        if frame_num not in frames_dict:
            continue

        kps = frames_dict[frame_num]['keypoints']
        if wrist_idx >= len(kps):
            continue

        wrist_y = kps[wrist_idx][1]

        if prev_y is not None:
            delta_y = prev_y - wrist_y  # Positive = upward
            if delta_y > 0:
                # Wrist is moving upward
                no_upward_motion = 0
            else:
                # Wrist has stopped rising
                no_upward_motion += 1

        if no_upward_motion >= still_frame_thresh:
            prep_frame = frame_num
            break

        prev_y = wrist_y

    return prep_frame


In [ ]:
def calculate_elbow_angle(keypoints, shoulder_idx=12, elbow_idx=14, wrist_idx=16):
    """Calculates elbow angle (shoulder-elbow-wrist) in degrees"""

    shoulder = keypoints[shoulder_idx][:2]
    elbow = keypoints[elbow_idx][:2]
    wrist = keypoints[wrist_idx][:2]

    # Vectors from elbow to shoulder and wrist
    vec_se = np.array(shoulder) - np.array(elbow)
    vec_ew = np.array(wrist) - np.array(elbow)

    # Calculate angle
    cosine = np.dot(vec_se, vec_ew) / (np.linalg.norm(vec_se) * np.linalg.norm(vec_ew))
    angle = np.degrees(np.arccos(np.clip(cosine, -1, 1)))
    return angle

In [ ]:
def generate_elbow_angle_sequence(frames_dict, prep_frame, release_frame, lefty=False):
    """
    Extracts a sequence of elbow angles from prep to release using calculate_elbow_angle().

    Returns:
        List of elbow angles (float) per frame
    """
    # COCO keypoint indices: shoulder, elbow, wrist
    if lefty:
        shoulder_idx, elbow_idx, wrist_idx = 5, 7, 9
    else:
        shoulder_idx, elbow_idx, wrist_idx = 6, 8, 10

    angle_sequence = []

    for frame_num in range(prep_frame, release_frame + 1):
        if frame_num not in frames_dict:
            continue

        keypoints = frames_dict[frame_num]['keypoints']

        if any(idx >= len(keypoints) for idx in [shoulder_idx, elbow_idx, wrist_idx]):
            continue

        angle = calculate_elbow_angle(
            keypoints,
            shoulder_idx=shoulder_idx,
            elbow_idx=elbow_idx,
            wrist_idx=wrist_idx
        )

        angle_sequence.append(angle)

    return angle_sequence


In [ ]:
def generate_knee_angle_sequence(frames_dict, prep_frame, release_frame, lefty=False):
    """
    Extracts a sequence of knee angles from prep to release.

    Returns:
        List of knee angles (float) per frame
    """
    # COCO indices for hips, knees, ankles
    if lefty:
        hip_idx, knee_idx, ankle_idx = 11, 13, 15  # left leg
    else:
        hip_idx, knee_idx, ankle_idx = 12, 14, 16  # right leg

    angle_sequence = []

    for frame_num in range(prep_frame, release_frame + 1):
        if frame_num not in frames_dict:
            continue

        keypoints = frames_dict[frame_num]['keypoints']

        if any(idx >= len(keypoints) for idx in [hip_idx, knee_idx, ankle_idx]):
            continue

        angle = calculate_knee_angle(keypoints)  # assumes your function uses COCO convention internally
        angle_sequence.append(angle)

    return angle_sequence

In [ ]:


def get_ball_center_from_bbox(bbox):
    """
    Returns the center (x, y) of a bounding box.
    Accepts either (x1, y1, x2, y2) or (x, y, w, h).
    """
    if len(bbox) == 4:
        if bbox[2] > bbox[0]:  # Assume (x1, y1, x2, y2)
            x1, y1, x2, y2 = bbox
            return ((x1 + x2) / 2, (y1 + y2) / 2)
        else:  # Assume (x, y, w, h)
            x, y, w, h = bbox
            return (x + w / 2, y + h / 2)
    else:
        raise ValueError("Invalid bbox format")


def calculate_ball_trajectory_on_release(frames_dict, release_frame, max_lookahead=5):
    """
    Calculates the ball's upward trajectory angle at release using the release frame
    and the next frame with a detected ball bounding box (within a given lookahead range).

    Args:
        frames_dict (dict): Dictionary of frame data with 'ball_bbox' entries.
        release_frame (int): Frame index of the shot release.
        max_lookahead (int): Maximum number of frames to look ahead for a valid detection.

    Returns:
        float or None: Upward trajectory angle in degrees, or None if insufficient data.
    """

    def get_ball_center_from_bbox(bbox):
        if len(bbox) == 4:
            if bbox[2] > bbox[0]:  # (x1, y1, x2, y2)
                x1, y1, x2, y2 = bbox
                return ((x1 + x2) / 2, (y1 + y2) / 2)
            else:  # (x, y, w, h)
                x, y, w, h = bbox
                return (x + w / 2, y + h / 2)
        return None

    if release_frame not in frames_dict:
        return None

    ball_bbox_current = frames_dict[release_frame].get('ball_bbox')
    if ball_bbox_current is None:
        return None

    center_current = get_ball_center_from_bbox(ball_bbox_current)

    # Look forward for next frame with a valid ball detection
    next_frame = None
    center_next = None
    for offset in range(1, max_lookahead + 1):
        f = release_frame + offset
        if f in frames_dict:
            ball_bbox_next = frames_dict[f].get('ball_bbox')
            if ball_bbox_next is not None:
                center_next = get_ball_center_from_bbox(ball_bbox_next)
                next_frame = f
                break

    if center_next is None:
        return None  # No valid future detection found

    dx = center_next[0] - center_current[0]
    dy = center_current[1] - center_next[1]  # image space: upward is negative

    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)

    return round(angle_deg, 2)

In [ ]:
def generate_dtw_scores_and_angles(student_keypoints, teacher="SGA", is_lefty=False):

  keypoints_path = '/content/ComputingProject/TeacherSequences/'+teacher+'_keypoints.pkl'
  with open(keypoints_path, 'rb') as f:
    teacher_keypoint_data = pickle.load(f)

  teacher_lefty = False

  student_release_frame, student_release_angle = find_release_frame(student_keypoints)
  student_load_frame, student_load_angle, student_knee_angle = find_load_frame(student_keypoints, student_release_frame, y_tolerance=5)
  student_prep_frame = find_preparation_frame(student_keypoints, student_load_frame, lefty=is_lefty)

  teacher_release_frame, teacher_release_angle = find_release_frame(teacher_keypoint_data)
  teacher_load_frame, teacher_load_angle, teacher_knee_angle  = find_load_frame(teacher_keypoint_data, teacher_release_frame, y_tolerance=5)
  teacher_prep_frame = find_preparation_frame(teacher_keypoint_data, teacher_load_frame, lefty=teacher_lefty)

  student_elbows = generate_elbow_angle_sequence(student_keypoints, student_prep_frame, student_release_frame, lefty=is_lefty)
  teacher_elbows = generate_elbow_angle_sequence(teacher_keypoint_data, teacher_prep_frame, teacher_release_frame, lefty=teacher_lefty)

  student_knees = generate_knee_angle_sequence(student_keypoints, student_prep_frame, student_release_frame, lefty=is_lefty)
  teacher_knees = generate_knee_angle_sequence(teacher_keypoint_data, teacher_prep_frame, teacher_release_frame, lefty=teacher_lefty)

  elbow_score = compare_angle_sequences_dtw(student_elbows, teacher_elbows)
  knee_score = compare_angle_sequences_dtw(student_knees, teacher_knees)

  student_trajectory_angle = calculate_ball_trajectory_on_release(student_keypoints, student_release_frame)
  teacher_trajectory_angle = calculate_ball_trajectory_on_release(teacher_keypoint_data, teacher_release_frame)


  return elbow_score, knee_score, teacher_load_angle, teacher_knee_angle, teacher_release_angle, student_trajectory_angle, teacher_trajectory_angle

In [ ]:
analyse_video('/content/ComputingProject/inputVideos/FT_Train2.mp4')

In [ ]:
def visualise_shooter_with_ball_trajectory(frames_dict, video_path, output_path, release_frame=None):
    """
    Creates visualisation with pose estimation and ball trajectory
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video {video_path}")

    # Get video properties directly from file (ensures compatibility)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Video writer with same parameters as track_shooter_full_video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Ball trajectory tracking
    trajectory_points = []
    release_y = None
    ball_color = (0, 255, 0)  # BGR format (green)
    trajectory_color = (255, 0, 0)  # BGR format (red)

    # Get the first relevant frame from frames_dict
    start_frame = min(frames_dict.keys())

    # Sync the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)


    for frame_num in sorted(frames_dict.keys()):
        ret, frame = cap.read()
        if not ret:
            break

        frame_data = frames_dict[frame_num]

        # 1. Create pose visualization
        vis_frame = frame.copy()
        if frame_data['keypoints'] is not None:
            # Convert to MMPose format
            vis_keypoints = np.array([frame_data['keypoints']])  # x,y only
            vis_scores = np.array([frame_data['keypoint_scores']])

            # Generate visualisation
            metainfo = '/content/ComputingProject/mmpose/configs/_base_/datasets/coco.py'
            combined_img = visualize(
                cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB),
                vis_keypoints,
                vis_scores,
                metainfo=metainfo,
                show=False
            )

            # Extract right half for keypoints visualisation (critical for dimensions)
            height_temp, width_temp = combined_img.shape[:2]
            vis_frame = combined_img[:, width_temp//2:]

        # 2. Add ball trajectory (same as original tracking function)
        if frame_data.get('ball_bbox'):
            x1, y1, x2, y2 = frame_data['ball_bbox']
            ball_center = (int((x1+x2)/2), int((y1+y2)/2))

            # Initialise release height
            if frame_num == release_frame:
                release_y = ball_center[1]
                release_y_padded = release_y + 100
                trajectory_points = [ball_center]
            elif release_y is not None and ball_center[1] <= release_y_padded:
                trajectory_points.append(ball_center)

            # Draw trajectory lines
            for i in range(1, len(trajectory_points)):
                cv2.line(vis_frame, trajectory_points[i-1], trajectory_points[i],
                        trajectory_color, 2)

            # Draw current ball position
            cv2.circle(vis_frame, ball_center, 8, ball_color, -1)

        # 3. Write frame (ensuring correct dimensions)
        out.write(cv2.resize(vis_frame, (width, height)))

    cap.release()
    out.release()
    return output_path

In [ ]:
def find_release_frame(frames_dict, max_ball_distance=80, top_n=5, lefty=False):
    """
    iterative batch-wise release frame finder:
    - Processes top-N wrist heights in batches
    - For each frame: ball proximity first, fallback velocity check if no ball exists.
    """

    import numpy as np
    from scipy.signal import savgol_filter

    if lefty:
        shoulder_idx, elbow_idx, wrist_idx = 5, 7, 9
    else:
        shoulder_idx, elbow_idx, wrist_idx = 6, 8, 10

    all_wrist_y = []
    frame_nums = []
    frame_data = {}

    # Preprocess all frames
    for frame_num, data in frames_dict.items():
        keypoints = data.get('keypoints')
        if keypoints is not None:
            wrist_pos = keypoints[wrist_idx][:2]
            all_wrist_y.append(wrist_pos[1])
            frame_nums.append(frame_num)
            frame_data[frame_num] = {
                'wrist_y': wrist_pos[1],
                'keypoints': keypoints,
                'ball_bbox': data.get('ball_bbox')
            }

    # Smooth wrist positions
    y_array = np.array(all_wrist_y)
    smoothed_y = savgol_filter(y_array, 5, 2)

    # Sort all frames by wrist height (lowest Y highest position)
    sorted_indices = np.argsort(smoothed_y)
    sorted_frames = [frame_nums[idx] for idx in sorted_indices]

    # Iterate through sorted frames in batches of top_n
    for batch_start in range(0, len(sorted_frames), top_n):
        batch_frames = sorted_frames[batch_start: batch_start + top_n]

        for candidate_frame in batch_frames:
            candidate_data = frame_data[candidate_frame]
            ball_bbox = candidate_data['ball_bbox']

            # Ball proximity check first
            if ball_bbox is not None:
                x1, y1, x2, y2 = ball_bbox
                ball_center = [(x1 + x2) / 2, (y1 + y2) / 2]
                wrist_pos = candidate_data['keypoints'][wrist_idx][:2]
                dist_to_ball = np.linalg.norm(wrist_pos - np.array(ball_center))

                if dist_to_ball <= max_ball_distance:
                    release_angle = calculate_elbow_angle(candidate_data['keypoints'], shoulder_idx, elbow_idx, wrist_idx)
                    return candidate_frame, release_angle

            # Fallback velocity check if no ball detection
            elif ball_bbox is None:
                idx_in_full = frame_nums.index(candidate_frame)
                if idx_in_full < 3:
                    continue

                prior_y_vals = smoothed_y[idx_in_full - 3: idx_in_full + 1]
                velocity = np.gradient(prior_y_vals)
                avg_upward = -np.mean(velocity[:-1])

                if avg_upward >= 3.0:  # threshold
                    release_angle = calculate_elbow_angle(candidate_data['keypoints'], shoulder_idx, elbow_idx, wrist_idx)
                    return candidate_frame, release_angle

    # If all batches exhausted
    return None


In [ ]:
def crop_shooter(shooter_keypoints_data, frame_num, video_path, padding_ratio=0.3,target_size=256, paint_joints=[]):
  """
  crops shooter for specified frame
  """
  cap = cv2.VideoCapture(video_path)
  cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
  ret, frame = cap.read()
  frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

  current_shooter_bbox = shooter_keypoints_data[frame_num]['bbox']
  keypoints = shooter_keypoints_data[frame_num]['keypoints']
  x1, y1, x2, y2 = current_shooter_bbox
  w, h = x2 - x1, y2 - y1

  # Calculate square crop with padding
  size = max(w, h) * (1 + padding_ratio)
  center_x, center_y = (x1 + x2)/2, (y1 + y2)/2

  # Get crop coordinates
  crop_x1 = max(0, int(center_x - size/2))
  crop_y1 = max(0, int(center_y - size/2))
  crop_x2 = min(frame.shape[1], int(center_x + size/2))
  crop_y2 = min(frame.shape[0], int(center_y + size/2))



  # Draw wrist marker on original frame (before cropping)
  if paint_joints is not None:
    for i in paint_joints:
      joint_x, joint_y = keypoints[i][:2]
      cv2.circle(frame_rgb, (int(joint_x), int(joint_y)), 5, (0, 255, 0), -1)  # Green dot


  # Handle edge cases by adjusting opposite side
  if crop_x1 < 0:
      crop_x2 += abs(crop_x1)
      crop_x1 = 0
  if crop_y1 < 0:
      crop_y2 += abs(crop_y1)
      crop_y1 = 0
  if crop_x2 > frame.shape[1]:
      crop_x1 -= (crop_x2 - frame.shape[1])
      crop_x2 = frame.shape[1]
  if crop_y2 > frame.shape[0]:
      crop_y1 -= (crop_y2 - frame.shape[0])
      crop_y2 = frame.shape[0]

  # Final crop and resize
  crop = frame_rgb[crop_y1:crop_y2, crop_x1:crop_x2]
  resized = cv2.resize(crop, (target_size, target_size))
  return(resized)

In [ ]:


def compare_angle_sequences_dtw(seq1, seq2):
    """
    Computes DTW distance between two sequences of joint angles.

    Args:
        seq1: List of angles (e.g., student)
        seq2: List of angles (e.g., reference)

    Returns:
        float: DTW distance score (lower = more similar)
    """
    # Convert to 2D arrays for DTW (shape: (n, 1))
    seq1 = np.array(seq1).reshape(-1, 1)
    seq2 = np.array(seq2).reshape(-1, 1)

    distance, _, _, path = dtw(seq1, seq2, dist=lambda x, y: np.abs(x - y))
    normalised_score = (distance / len(path[0]))
    return normalised_score

def interpret_dtw_score(score):
  if score < 5:
      return "Excellent – movement closely matches the reference."
  elif score < 10:
      return "Good – minor inconsistencies in form."
  elif score < 20:
      return "Moderate – noticeable form variations."
  else:
      return "Needs Improvement – substantial deviation from reference technique."

def generate_joint_recommendations(student_angle, teacher_angle, joint_name):
    diff = student_angle - teacher_angle
    if abs(diff) < 5:
        return f"Your {joint_name} angle is well aligned with the reference."
    elif diff > 0:
        return f"Try reducing your {joint_name} angle slightly during the shot for a more compact form."
    else:
        return f"Consider increasing your {joint_name} angle to better mirror the reference motion."

def generate_trajectory_recommendation(student_angle, teacher_angle):
    if student_angle is None or teacher_angle is None:
        return "No trajectory data available for comparison."
    diff = student_angle - teacher_angle
    if abs(diff) < 3:
        return "✅ Your shot arc closely matches the reference — well done!"
    elif diff > 0:
        return "⬆️ Your shot arc is higher than the reference. This may improve forgiveness, but watch for loss of power or overcompensation."
    else:
        return "⬇️ Your shot arc is flatter than the reference. Consider increasing your upward wrist motion for a softer, more controlled release."


def build_recommendation_section(elbow_score, knee_score, student_angles, teacher_angles):
    load_elbow_student, load_knee_student, release_elbow_student, trajectory_s = student_angles
    load_elbow_teacher, load_knee_teacher, release_elbow_teacher, trajectory_t = teacher_angles

    elbow_band = interpret_dtw_score(elbow_score)
    knee_band = interpret_dtw_score(knee_score)

    elbow_tip = generate_joint_recommendations(load_elbow_student, load_elbow_teacher, "elbow (load phase)")
    knee_tip = generate_joint_recommendations(load_knee_student, load_knee_teacher, "knee (load phase)")
    release_tip = generate_joint_recommendations(release_elbow_student, release_elbow_teacher, "elbow (release phase)")
    traj_tip = generate_trajectory_recommendation(trajectory_s, trajectory_t)

    if trajectory_s is None:
      trajectory_s = "N/A"
    else:
      trajectory_s = f"{trajectory_s:.1f}"


    return (
        f"### 🔍 Recommendations\n"
        f"**Elbow Form (DTW Score: {elbow_score:.2f})** — {elbow_band}\n"
        f"{elbow_tip}\n"
        f"{release_tip}\n\n"
        f"**Knee Form (DTW Score: {knee_score:.2f})** — {knee_band}\n"
        f"{knee_tip}\n\n"
        f"**Ball Trajectory**\n"
        f"Student: {trajectory_s:}° | Reference: {trajectory_t:.1f}°\n"
        f"{traj_tip}"
    )


In [ ]:

def analyse_video(video_path, is_lefty=False, chosen_teacher="SGA"):

  # progess message yield
  yield None, None, None, "", "", "🔄 Initialising models..."
  # Initialise models
  yolo_model = YOLO('/content/ComputingProject/yolov11/best.pt')

  mmpose_inferencer_2d = MMPoseInferencer('human')

  weights_3d = '/content/ComputingProject/mmpose/checkpoints/videopose_h36m_1frame_fullconv_supervised_cpn_ft-5c3afaed_20210527.pth'
  config_3d = '/content/ComputingProject/mmpose/configs/body_3d_keypoint/video_pose_lift/h36m/video-pose-lift_tcn-1frm-supv-cpn-ft_8xb128-160e_h36m.py'
  mmpose_inferencer_3d = MMPoseInferencer(
  pose3d=config_3d,
  pose3d_weights=weights_3d,
  device='cuda:0'
  )

  if isinstance(video_path, str):
        # Regular file path (testing)
        input_path = video_path
        # Create temp output file
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as output_tmp:
            output_path = output_tmp.name
  else:
        # Gradio file object (deployment)
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as input_tmp, \
            tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as output_tmp:

            # Save uploaded video
            with open(input_tmp.name, "wb") as f:
                f.write(video_path.read())
            input_path = input_tmp.name
            output_path = output_tmp.name


  yield None, None, None, "", "", "📹 Tracking shooter and extracting keypoints..."
  # start video processing
  is_lefty = (hand_radio == "Left-handed")
  keypoints_data = track_shooter_full_video(input_path, yolo_model, mmpose_inferencer_2d)

  yield None, None, None, "", "", "📌 Identifying shot phases..."

  release_frame, release_angle = find_release_frame(keypoints_data, lefty=is_lefty)
  load_frame, load_angle, knee_angle=find_load_frame(keypoints_data, release_frame, lefty=is_lefty)

  # Generate outputs visualisations
  yield None, None, None, "", "", "🎯 Visualising 2D output..."

  visualise_shooter_with_ball_trajectory(
      frames_dict=keypoints_data,
      video_path=input_path,
      output_path=output_path,
      release_frame=release_frame,
  )

  yield None, None, None, "", "", "🦾 Running 3D lifting..."
  # Cropping Key Phases for 3d lifting
  release_crop = crop_shooter(keypoints_data, release_frame, input_path)
  load_crop = crop_shooter(keypoints_data, load_frame, input_path)

  # 3D lifting
  release_dict = next(mmpose_inferencer_3d(release_crop, return_vis=True, num_instances = 1))
  load_dict = next(mmpose_inferencer_3d(load_crop, return_vis=True, num_instances = 1))

  release_vis = cv2.cvtColor(release_dict['visualization'][0], cv2.COLOR_BGR2RGB)
  load_vis = cv2.cvtColor(load_dict['visualization'][0], cv2.COLOR_BGR2RGB)
  release_vis = Image.fromarray(release_vis)
  load_vis = Image.fromarray(load_vis)

  yield None, None, None, "", "", "📊 Calculating DTW scores..."

  # dtw score calculation
  elbow_score, knee_score, teacher_load_angle, teacher_knee_angle, teacher_release_angle, student_trajectory_angle, \
   teacher_trajectory_angle = generate_dtw_scores_and_angles(keypoints_data, is_lefty=is_lefty, teacher=chosen_teacher)

  recommendation_section = build_recommendation_section(
  elbow_score, knee_score,
  student_angles=(load_angle, knee_angle, release_angle, student_trajectory_angle),
  teacher_angles=(teacher_load_angle, teacher_knee_angle, teacher_release_angle, teacher_trajectory_angle))

  if student_trajectory_angle is None:
      student_trajectory_angle = "N/A"
  else:
      student_trajectory_angle = f"{student_trajectory_angle:.1f}"



  summary = (
    f"### 📊 DTW Similarity Scores\n"
    f"- Elbow: {elbow_score:.2f}\n"
    f"- Knee: {knee_score:.2f}\n\n"
    f"### 🧍‍♂️ Joint Angles (Student vs {chosen_teacher})\n"
    f"**Load Phase:**\n"
    f"- Elbow: {load_angle:.1f}° / {teacher_load_angle:.1f}°\n"
    f"- Knee: {knee_angle:.1f}° / {teacher_knee_angle:.1f}°\n"
    f"**Release Phase:**\n"
    f"- Elbow: {release_angle:.1f}° / {teacher_release_angle:.1f}°\n\n"
    f"### 🏀 Ball Trajectory at Release\n"
    f"- Student: {student_trajectory_angle}°\n"
    f"- {chosen_teacher}: {teacher_trajectory_angle:.1f}°\n"
  )


  yield output_path, release_vis, load_vis, summary, recommendation_section, "✅ Analysis complete!"

In [ ]:
### GUI INITIALISATION ###

with gr.Blocks() as app:
    gr.Markdown("## 🏀 Basketball Shot Analyser")

    # Progress bar at the top
    progress_text = gr.Textbox(
        label="Progress",
        lines=2,
        max_lines=2,
        interactive=False
    )

    # upload video and other input args
    with gr.Row():
        with gr.Column(scale=2):
            video_input = gr.Video(label="Upload Shot Video", height=320)
        with gr.Column(scale=1):
            hand_radio = gr.Radio(
                ["Right-handed", "Left-handed"],
                label="Shooter's Dominant Hand",
                value="Right-handed"
            )
            teacher_dropdown = gr.Dropdown(
                ["SGA", "Vanfleet"],
                label="Select Reference Player (Teacher)",
                value="SGA"
            )
            submit_btn = gr.Button("Analyse Shot", variant="primary")

    # 3d poses and 2d vis
    with gr.Row():
        with gr.Column(scale=2):
            video_output = gr.Video(label="Processed Video with Ball Trajectory", height=320)
        with gr.Column(scale=1):
            with gr.Row():
                release_img = gr.Image(label="3D Pose: Release Frame", type="pil", height=150)
            with gr.Row():
                load_img = gr.Image(label="3D Pose: Load Frame", type="pil", height=150)

    # analysis and recommendation text boxes
    with gr.Row():
      with gr.Column(scale=1):
          analysis_summary = gr.Textbox(
              label="📊 Analysis Summary",
              lines=10,
              max_lines=20,
              interactive=False,
              show_copy_button=True
          )
      with gr.Column(scale=1):
          recommendation_summary = gr.Textbox(
              label="🔍 Recommendations",
              lines=10,
              max_lines=20,
              interactive=False,
              show_copy_button=True
          )

    # Button wiring — `analyse_video` yields results directly
    submit_btn.click(
        fn=analyse_video,
        inputs=[video_input, hand_radio, teacher_dropdown],
        outputs=[video_output, release_img, load_img, analysis_summary, recommendation_summary, progress_text]
    )

app.launch(debug=True)
